In [ ]:
############ LOAD in custom packages ################
import sys
import os
import pandas as pd
import numpy as np
import importlib
import multiprocessing as mp
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import xarray as xr
import numpy as np

# project_dir = '/Users/demo/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2'
# sub_logs = os.path.join(project_dir, 'subject_logs')

# subs = {}
# for sub in os.listdir(sub_logs):
#     if 'sub-' in sub:
#         parts = sub.split('.')
#         sub_id = parts[0]
#         subs[sub_id] = pd.read_excel(os.path.join(sub_logs, sub),sheet_name='JSON_formatted')

import sys
import os
import pandas as pd
import datetime
from datetime import datetime as dt
import plotly.express as px

def extract_survey_date(filename: str):
	"""
	Extracts a survey date from a filename of the form '..._<Month Day, Year>.csv'
	Example: 'survey_results_Sep 06, 2025.csv' → datetime(2025, 9, 6)
	"""
	try:
		# Grab the last part after the last underscore, before the extension
		datestr = filename.split('_')[-1].split('.')[0]
		return dt.strptime(datestr, '%b %d, %Y')
	except Exception:
		return None
	
def get_most_recent_survey(directory, recoded=False):
	# Allowed Excel extensions
	extensions = (".csv", ".xls", ".xlsx")
	filenames = [f for f in os.listdir(directory) if f.endswith(extensions)]
	if recoded==True:
		filenames = [f for f in filenames if 'recoded' in f]
	else:
		filenames = [f for f in filenames if not 'recoded' in f]
	
	# List Excel files with their sizes
	dates = []
	for filename in filenames:

		survey_date = extract_survey_date(filename)
		if survey_date is not None:
			dates.append({'filename': filename, 'date': survey_date})
			

	if len(dates) > 0:
		dates.sort(key=lambda file: file['date'], reverse=True)

		return os.path.join(directory, dates[0]['filename'])
	
	return None



data_dir = os.path.expanduser('~/Library/CloudStorage/Box-Box/(Restricted)_PCR/PCX')

# project where survey exports live
surveys_dir = os.path.join(data_dir, 'behavioral')

# folder where your fmriprep reports live
REPORTS_DIR = os.path.expanduser('~/Library/CloudStorage/Box-Box/(Restricted)_PCR/PCX/fmriprep_reports')
tracker_df=pd.read_excel(os.path.expanduser('~/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Subject_tracker_PCR.xlsx'), sheet_name='tracker')


survey_names = ['clinical_administered_data','clinical_self_report_data',
					 'mri_self_report_data','supplemental_self_report_data']
subsurvey_key = {
	'panss': 'clinical_administered_data',
	'madrs': 'clinical_administered_data',
	'ymrs': 'clinical_administered_data',
	'bprs': 'clinical_administered_data',
	'cssrs': 'clinical_administered_data',

}

subject_id_pattern = r"^qual[rm]2\d{2}$"

surveys={}
recoded_surveys={}
filepath=None
for root, dirs, files in os.walk(surveys_dir):
	for survey in survey_names: 
		if survey in dirs: 
			surveys[survey] = []
			survey_dir = os.path.join(root, survey)
			try:
				filepath = get_most_recent_survey(survey_dir)
				filepath_recoded = get_most_recent_survey(survey_dir, recoded=True)
			except Exception as e:
				print(e)
			if filepath is not None:
				surveys[survey] = pd.read_csv(filepath)
				surveys[survey]['SUBJECT_ID'] = surveys[survey]['SUBJECT_ID'].str.lower()
				surveys[survey]['SUBJECT_ID'] = surveys[survey]['SUBJECT_ID'].str.replace('pc','qual')

				mask = surveys[survey]['SUBJECT_ID'].astype(str).str.strip().str.match(subject_id_pattern, na=False) # boolean mask for valid subject IDs 
				valid_df = surveys[survey][mask] # keep only matching
				# print skipped subject IDs
				skipped = surveys[survey][~mask]
				print(f"{survey} Skipped rows:",skipped['SUBJECT_ID'].tolist())
				surveys[survey] = valid_df

				surveys[survey][survey] = surveys[survey]['StartDate']
				surveys[survey]['SUBJECT_ID'] = surveys[survey]['SUBJECT_ID'].str.lower()

				surveys[survey] = surveys[survey]
	
				recoded_surveys[survey]=pd.read_csv(filepath_recoded)
				recoded_surveys[survey]['SUBJECT_ID'] = recoded_surveys[survey]['SUBJECT_ID'].str.lower()
				recoded_surveys[survey]['SUBJECT_ID'] = recoded_surveys[survey]['SUBJECT_ID'].str.replace('pc','qual')
				mask = recoded_surveys[survey]['SUBJECT_ID'].astype(str).str.match(subject_id_pattern, na=False) # boolean mask for valid subject IDs 
				valid_df = recoded_surveys[survey][mask] # keep only matching
				recoded_surveys[survey] = valid_df
				recoded_surveys[survey][str(survey)] = recoded_surveys[survey]['StartDate']
				recoded_surveys[survey] = recoded_surveys[survey]



rmr_df = pd.read_excel(os.path.expanduser('~/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Admin/RMR/RMR_running.xlsx'))
tracker_df=pd.read_excel(os.path.expanduser('~/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Subject_tracker_PCR.xlsx'), sheet_name='tracker')
subs_df=pd.read_excel(os.path.expanduser('~/Library/CloudStorage/Box-Box/Holmes_Lab_Wiki/PCX_Round2/Subject_tracker_PCR.xlsx'), sheet_name='tracker')



clinical_administered_data Skipped rows: ['subject id (ie qualr200 or qualm200)', '{"importid":"qid128_text"}', 'qualryk', 'qualralex_test', nan, nan, nan, nan, 'qualr_kaley testing', 'qualrkaleytest', 'qualrtestlex']
clinical_self_report_data Skipped rows: ['subject id (ie qualr200 or qualm200, ask the clinician for this id number)', '{"importid":"qid96_text"}', 'qualr217 ', 'test', 'test', 'qualm211\n1']
mri_self_report_data Skipped rows: ['subject id (ie qualr200 or qualm200, should be told to you by study staff)', '{"importid":"qid165_text"}']
supplemental_self_report_data Skipped rows: ['subject id (starts with "qual", for example, "qualr200". you can find this it the email where this survey link was sent to you)', '{"importid":"qid100_text"}', 'qual207', 'qual']


In [79]:


# num phone data: in accel_df, gps_df, power_df
# num mri data: in surveys['mri_self_report_data']
# num clin interview session: in surveys['clinical_administered_data']
# goal: in rmr

 #RMR visual

# Find primary and other diagnoses, reshape them into list
session1 = surveys['clinical_administered_data']
prim_diagnoses_cols = ['SUBJECT_ID']+[col for col in session1.columns if 'primary_diagnoses' in col]
session1['primary_diagnoses_all'] = session1[prim_diagnoses_cols].bfill(axis=1).iloc[:, 1:2]
other_diagnoses_cols = ['SUBJECT_ID']+[col for col in session1.columns if 'other' in col]
session1['other_diagnoses_all'] = session1[other_diagnoses_cols].bfill(axis=1).iloc[:, 1:2]
session1['duration_mins'] = pd.to_numeric(session1['Duration (in seconds)'])/60
session1_r = session1[session1['SITE_ID']=='Rutgers University / UBHC']
session1_m = session1[session1['SITE_ID'] == 'McLean Hospital']
session1_clean = session1.dropna(subset='SUBJECT_ID')
session1sr = surveys['clinical_self_report_data']
session1sr['SUBJECT_ID'] = session1sr['SUBJECT_ID'].str.lower()
session1sr['SUBJECT_ID'] = session1sr['SUBJECT_ID'].str.replace('pcr', 'qualr')
session1sr = session1sr[~session1sr['SUBJECT_ID'].str.contains('test', na=False)]
session1sr_clean = session1sr.dropna(subset='SUBJECT_ID')

session1sr_clean['duration_mins'] = pd.to_numeric(session1sr_clean['Duration (in seconds)'])/60
session1sr_r = session1sr_clean[session1sr_clean['SITE_ID']=='Rutgers University / UBHC']
session1sr_m = session1sr_clean[session1sr_clean['SITE_ID'] == 'McLean Hospital']
session2 = surveys['mri_self_report_data']
session2['SUBJECT_ID'] = session2['SUBJECT_ID'].str.lower()
session2['SUBJECT_ID'] = session2['SUBJECT_ID'].str.replace('pcr', 'qualr')
session2 = session2[~session2['SUBJECT_ID'].str.contains('test', na=False)]

session2_clean = session2.dropna(subset='SUBJECT_ID')

session2['duration_mins'] = pd.to_numeric(session2_clean['Duration (in seconds)'])/60
session2_r = session2_clean[session2_clean['SITE_ID']=='Rutgers University / UBHC']
session2_m = session2_clean[session2_clean['SITE_ID'] == 'McLean Hospital']
total_real = len(session1_r) + len(session1_m)

# Pie charts
primary_pie = px.pie(session1, names='primary_diagnoses_all', title='Subject Primary Diagnoses')
other_pie = px.pie(session1, names='other_diagnoses_all', title='Subject Other Diagnoses')


survey_cols = [
    "SUBJECT_ID",'SITE_ID','primary_diagnoses_all','other_diagnoses_all',
    'clinical_administered_data','clinical_self_report_data', 
    'mri_self_report_data','supplemental_self_report_data',
	"sex","age", "ethnic","racial","place_birth", 'name_meds','purpose_meds']

session0_merge = session1[[col for col in survey_cols if col in session1.columns]]
session1_merge = session1sr[[col for col in survey_cols if col in session1sr.columns]]
session2_merge = session2_clean[[col for col in survey_cols if col in session2_clean.columns]]

session3 = surveys['supplemental_self_report_data']
session3_clean = session3.dropna(subset='SUBJECT_ID')
session3_merge = session3_clean[[col for col in survey_cols if col in session3_clean.columns]]
session3_merge['SUBJECT_ID'] = session3_merge['SUBJECT_ID'].str.lower()



# align on SUBJECT_ID first
s1 = session1_merge.set_index("SUBJECT_ID")
s2 = session2_merge.set_index("SUBJECT_ID")
s3 = session3_merge.set_index("SUBJECT_ID")
s0 = session0_merge.set_index("SUBJECT_ID")
demographic_df = s1.combine_first(s2).combine_first(s3).combine_first(s0).reset_index()


demographic_df = (
    demographic_df.groupby("SUBJECT_ID")
    .agg(lambda x: ", ".join(x.dropna().astype(str).unique()))
    .reset_index()
)
display(demographic_df[survey_cols])


/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_87030/1883369596.py:11: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_87030/1883369596.py:13: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_87030/1883369596.py:57: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,SUBJECT_ID,SITE_ID,primary_diagnoses_all,other_diagnoses_all,clinical_administered_data,clinical_self_report_data,mri_self_report_data,supplemental_self_report_data,sex,age,ethnic,racial,place_birth,name_meds,purpose_meds
0,qual,Rutgers,,,,,,2025-08-06 14:14:28,,,,,,,
1,qual207,Rutgers,,,,,,2025-07-14 17:46:18,,,,,,,
2,qualm200,McLean,Bipolar disorder,PTSD/Adjustment Disorder/Other specified traum...,2025-06-25 21:56:34,2025-06-25 21:56:28,2025-07-09 19:47:35,,Female,25,Not Hispanic or Latino,White,"Spartanburg, South Carolina, USA",lamictal,mood stabilizing
3,qualm204,McLean,Unipolar depression,Generalized anxiety disorder (GAD),2025-06-25 17:49:18,2025-06-25 17:49:13,2025-07-08 19:09:58,,Female,27,Not Hispanic or Latino,Black or African American,"Lawton, Oklahoma, USA",Sertraline,Depression
4,qualm208,McLean,Depression w/ psychotic features,PTSD/Adjustment Disorder/Other specified traum...,2025-07-22 17:10:03,2025-07-22 17:09:31,2025-07-30 19:09:15,,Female,61,Not Hispanic or Latino,White,meriden ct usa,lamotrigine,mood stabiliozer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,qualr_kaley testing,,,,2025-09-19 15:36:30,,,,,,,,,,
71,qualralex_test,,,,2025-09-19 16:23:03,,,,,,,,,,
72,qualrkaleytest,,Unipolar depression,Binge Eating,2025-09-23 18:36:26,,,,,,,,,,
73,qualrtestlex,Rutgers,Unipolar depression,Panic Disorder,2025-09-23 18:41:21,,,,,,,,,,


In [40]:



# Surveys
cad_recoded = recoded_surveys['clinical_administered_data']
panss_p_total_cols = [f'panss_p0{str(i)}' for i in range(1,8)]
panss_n_total_cols = [f'panss_n0{str(i)}' for i in range(1,8)]
panss_g_total_cols = [f'panss_g0{str(i)}' for i in range(1,8)]
bprs_total_cols = [f'bprs_0{str(i)}' for i in range(1,10)]+[f'bprs_1{i}' for i in range(0,9)]
ymrs_total_cols = [f'ymrs_0{str(i)}' for i in range(1,10)]+[f'ymrs_1{i}' for i in range(0,2)]
madrs_total_cols = [f'madrs_0{str(i)}' for i in range(1,10)]+['madrs_10']

all_cols = panss_p_total_cols+panss_n_total_cols+panss_g_total_cols+bprs_total_cols+ymrs_total_cols+madrs_total_cols
cad_recoded[all_cols] = cad_recoded[all_cols].astype(float)


cad_recoded['panss_p_total'] = cad_recoded[panss_p_total_cols].sum(axis=1)
cad_recoded['panss_n_total'] = cad_recoded[panss_n_total_cols].sum(axis=1)
cad_recoded['panss_g_total'] = cad_recoded[panss_g_total_cols].sum(axis=1)
cad_recoded['bprs_total'] = cad_recoded[bprs_total_cols].sum(axis=1)
cad_recoded['ymrs_total'] = cad_recoded[ymrs_total_cols].sum(axis=1)
cad_recoded['madrs_total'] = cad_recoded[madrs_total_cols].sum(axis=1)

display(cad_recoded)



KeyError: "None of [Index(['panss_p01', 'panss_p02', 'panss_p03', 'panss_p04', 'panss_p05',\n       'panss_p06', 'panss_p07', 'panss_n01', 'panss_n02', 'panss_n03',\n       'panss_n04', 'panss_n05', 'panss_n06', 'panss_n07', 'panss_g01',\n       'panss_g02', 'panss_g03', 'panss_g04', 'panss_g05', 'panss_g06',\n       'panss_g07', 'bprs_01', 'bprs_02', 'bprs_03', 'bprs_04', 'bprs_05',\n       'bprs_06', 'bprs_07', 'bprs_08', 'bprs_09', 'bprs_10', 'bprs_11',\n       'bprs_12', 'bprs_13', 'bprs_14', 'bprs_15', 'bprs_16', 'bprs_17',\n       'bprs_18', 'ymrs_01', 'ymrs_02', 'ymrs_03', 'ymrs_04', 'ymrs_05',\n       'ymrs_06', 'ymrs_07', 'ymrs_08', 'ymrs_09', 'ymrs_10', 'ymrs_11',\n       'madrs_01', 'madrs_02', 'madrs_03', 'madrs_04', 'madrs_05', 'madrs_06',\n       'madrs_07', 'madrs_08', 'madrs_09', 'madrs_10'],\n      dtype='object', name=0)] are in the [columns]"

In [54]:


# RMR visual
session1 = surveys['clinical_administered_data'][2:]
prim_diagnoses_cols = ['SUBJECT_ID']+[col for col in session1 if 'primary_diagnoses' in col]
session1['primary_diagnoses_all'] = session1[prim_diagnoses_cols].bfill(axis=1).iloc[:, 1:2]
other_diagnoses_cols = ['SUBJECT_ID']+[col for col in session1 if 'other_diagnoses' in col]
session1['other_diagnoses_all'] = session1[other_diagnoses_cols].bfill(axis=1).iloc[:, 1:2]
display(session1)

/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_65320/88481360.py:4: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_65320/88481360.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_65320/88481360.py:6: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavio

,Unnamed: 0,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,panss_n_total,panss_g_total,madrs_total,Create New Field or Choose From Dropdown...,ymrs_total,clinical_administered_data,primary_diagnoses,other_diagnoses,primary_diagnoses_all,other_diagnoses_all
2,4,2025-06-10 19:17:09,2025-06-10 20:12:09,IP Address,130.219.10.91,100,3299,True,2025-06-10 20:12:09,R_7QYoz5e9Fcg32we,...,33+3+33+2+2+33+33,2+2+2+33+33+2+33+33+33,1+1+0+2+1+1+1+1+10,NaN,1+2+0+1+0+0+0+0+0+0+0,1,Bipolar w/ psychotic features,Substance use disorder: Alcohol,Bipolar w/ psychotic features,Substance use disorder: Alcohol
3,5,2025-06-13 20:47:43,2025-06-13 21:20:12,IP Address,130.219.10.91,100,1948,True,2025-06-13 21:20:12,R_1CkgUUiDZ5ytQ4V,...,33+1+33+33+33+33+33,33+2+33+33+33+33+33+33+33,0+0+2+0+0+0+0+0+00,NaN,1+0+0+0+0+0+0+0+0+0+0,1,Healthy control,NaN,Healthy control,NaN
4,6,2025-06-13 21:36:47,2025-06-13 22:42:28,IP Address,67.81.84.35,100,3941,True,2025-06-13 22:42:28,R_5PaQpCvbznucVQK,...,33+1+33+33+2+33+33,2+3+33+33+33+33+33+33+33,0+0+0+0+0+0+0+0+00,NaN,0+0+0+0+0+0+0+0+0+0+0,1,Bipolar disorder,PTSD/Adjustment Disorder/Other specified traum...,Bipolar disorder,PTSD/Adjustment Disorder/Other specified traum...
5,7,2025-06-16 13:14:17,2025-06-16 14:04:54,IP Address,130.219.10.90,100,3036,True,2025-06-16 14:04:54,R_5bVR3zNWWwXNe3G,...,33+1+33+33+1+33+33,33+33+33+1+33+33+33+33+33,0+0+2+4+0+0+0+0+00,NaN,0+0+2+2+0+0+0+0+0+1+0,1,Schizoaffective Disorder,NaN,Schizoaffective Disorder,NaN
6,8,2025-06-16 16:38:03,2025-06-16 17:04:07,IP Address,130.219.10.91,100,1563,True,2025-06-16 17:04:08,R_6PAfu6k5XPnd6eJ,...,3+3+33+5+1+33+33,2+3+3+3+33+5+3+33+1,4+6+3+0+5+3+4+4+42,NaN,0+0+0+0+2+0+0+0+0+0+0,1,Unipolar depression,PTSD/Adjustment Disorder/Other specified traum...,Unipolar depression,PTSD/Adjustment Disorder/Other specified traum...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,70,2025-09-23 18:33:31,2025-09-23 18:33:41,IP Address,69.119.141.180,29,10,False,2025-09-30 18:33:44,R_7EntfZaxND6yVcz,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
80,70,2025-09-23 18:33:31,2025-09-23 18:33:41,IP Address,69.119.141.180,29,10,False,2025-09-30 18:33:44,R_7EntfZaxND6yVcz,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
81,71,2025-09-19 15:36:30,2025-09-23 18:33:58,IP Address,69.119.141.180,74,356247,False,2025-09-30 18:34:00,R_7zz1wIn5cpPtnr3,...,++++++,++++++++,++++2++++4,NaN,NaN,1,NaN,NaN,NaN,NaN
82,72,2025-09-23 18:36:26,2025-09-23 18:36:50,IP Address,69.119.141.180,50,24,False,2025-09-30 18:36:51,R_6JLcL5iHJpOSIqz,...,NaN,NaN,NaN,NaN,NaN,1,Unipolar depression,Binge Eating,Unipolar depression,Binge Eating


In [53]:
print(demographic_df.columns)

Index(['SUBJECT_ID', 'SITE_ID', 'age', 'clinical_self_report_data', 'ethnic',
       'house', 'live_with_whom', 'marital', 'mri_self_report_data',
       'native_lang', 'other_diagnoses', 'place_birth', 'primary_diagnoses',
       'racial', 'sex', 'supplemental_self_report_data', 'weight'],
      dtype='object')


In [ ]:
cad = surveys['clinical_administered_data']
prim_diagnoses_cols = ['SUBJECT_ID']+[col for col in cad if 'primary_diagnoses' in col]
primary = cad[prim_diagnoses_cols].bfill(axis=1).iloc[:, 0:2]
primary = primary.loc[~primary['SUBJECT_ID'].str.contains("{")]
primary = primary.loc[~primary['SUBJECT_ID'].str.contains("ie")]
primary = primary.dropna()
fig = px.pie(primary, names='primary_diagnoses_1')
fig.show()




In [26]:
other_diagnoses_cols = ['SUBJECT_ID']+[col for col in cad if 'other_diagnoses' in col]
other = cad[other_diagnoses_cols].bfill(axis=1).iloc[:, 0:2]
other = other.loc[~other['SUBJECT_ID'].str.contains("{")]
other = other.loc[~other['SUBJECT_ID'].str.contains("ie")]
other = other.dropna()
fig = px.pie(other, names='other_diagnoses_1')
fig.show()


In [78]:
today = datetime.datetime.today()
today_str = today.strftime('%b %d %Y')
start = 'Apr 1 2025'
rate_per_day = 0.15
beginning = dt.strptime(start, '%b %d %Y')
days_of_study = today - beginning
print(days_of_study)
today_goal = int(days_of_study.days * rate_per_day)
today_site = today_goal / 2
print(today_goal, today_site)
today_c = pd.DataFrame([today_str, np.nan, np.nan, today_site, today_site, today_goal, len(session1_r), len(session1_r), len(session2_r), len(session1_m), len(session1_m), len(session2_m), total_real])
today_row = today_c.T
today_row.columns = rmr_df.columns.to_list()
rmr_df_today = pd.concat([rmr_df, today_row])
rmr_df_today['Quarter'] = rmr_df_today['Quarter'].apply(lambda x: dt.strptime(x, '%b %d %Y'))
rmr_df_today = rmr_df_today.sort_values('Quarter').reset_index(drop=True)
rmr_df_today = rmr_df_today.reset_index()
display(rmr_df_today)
today_row = rmr_df_today.loc[rmr_df_today['Quarter']==today_str]
display(today_row)
today_index = today_row['index'].values.squeeze()


159 days, 17:08:46.912505
23 11.5


,index,Quarter,Participants per site,Rate,Rutgers Goal,McLean Goal,Total Goal,Rutgers Consented,Rutgers Clinical Interview,Rutgers Scan,McLean Consented,McLean Clinical Interview,McLean Scan,Total Real
0,0,2025-04-01,13,13/month,13,13,26,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2025-08-01,14,3.5/month,27,27,54,27.0,27.0,11.0,25.0,4.0,0.0,52.0
2,2,2025-09-07,NaN,NaN,11.5,11.5,23,49,49,22,9,9,7,58
3,3,2025-12-01,13,3.25/month,40,40,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2026-04-01,14,3.5/month,54,54,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,2026-08-01,13,3.25/month,67,67,134,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2026-12-01,14,3.5/month,81,81,162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2027-04-01,13,3.25/month,94,94,188,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,2027-08-01,14,3.5/month,108,108,216,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2027-12-01,13,3.25/month,121,121,242,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,index,Quarter,Participants per site,Rate,Rutgers Goal,McLean Goal,Total Goal,Rutgers Consented,Rutgers Clinical Interview,Rutgers Scan,McLean Consented,McLean Clinical Interview,McLean Scan,Total Real
2,2,2025-09-07,NaN,NaN,11.5,11.5,23,49,49,22,9,9,7,58


In [80]:
print(today_index)
two_quarters_out=today_index+3
print(two_quarters_out)
print(type(two_quarters_out))
rmr_df_today = rmr_df_today[:two_quarters_out]
display(rmr_df_today)

2
5
<class 'numpy.int64'>


,index,Quarter,Participants per site,Rate,Rutgers Goal,McLean Goal,Total Goal,Rutgers Consented,Rutgers Clinical Interview,Rutgers Scan,McLean Consented,McLean Clinical Interview,McLean Scan,Total Real
0,0,2025-04-01,13,13/month,13,13,26,9.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2025-08-01,14,3.5/month,27,27,54,27.0,27.0,11.0,25.0,4.0,0.0,52.0
2,2,2025-09-07,NaN,NaN,11.5,11.5,23,49,49,22,9,9,7,58
3,3,2025-12-01,13,3.25/month,40,40,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2026-04-01,14,3.5/month,54,54,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:


rmr_goal = px.line(rmr_df_today, x='Quarter', y=['Total Goal', 'Total Real'])
r_goal = px.line(rmr_df_today, x='Quarter', y=['Rutgers Goal', 'Rutgers Consented', 'Rutgers Clinical Interview', 'Rutgers Scan'])
m_goal = px.line(rmr_df_today, x='Quarter', y=['McLean Goal', 'McLean Consented', 'McLean Clinical Interview', 'McLean Scan'])

rmr_goal.show()
r_goal.show()
m_goal.show()

In [ ]:
px.defaults.height=600
df = pd.concat([sub[['key','value']].T for sub in subs.values()], axis=0, ignore_index=True)
df.columns = df.iloc[0]
df = df[df['id'] != 'id']
heatmap_columns = [col for col in df.columns if not 'notes' in col and not 'diagnosis' in col]
#heatmap_columns.remove('subject_id')
#display(df[heatmap_columns])
xr_data = xr.DataArray(df[heatmap_columns].values, 
					   dims=["rows", "columns"],
					   coords={"rows": df['id'], "columns": heatmap_columns})
xr_data_bin = xr_data.where(xr_data == 0, 1)
#print(xr_data_bin)
fig = px.imshow(xr_data_bin, origin='lower',
					title="Tasks Completed by Participants",
					zmin=0, zmax=1, color_continuous_scale=[[0, "white"], [1, "black"]],
					labels={ # replaces default labels by column name
					"rows": "Subject",  "columns": "Tasks Completed"}
					)
fig.show()
fig.write_html("dashboard.html") # interactive export 


In [17]:
# Diagnoses
cad = surveys['clinical_administered_data']
prim_diagnoses_cols = ['SUBJECT_ID']+[col for col in cad if 'primary_diagnoses' in col]
cad['primary_diagnoses_all'] = cad[prim_diagnoses_cols].bfill(axis=1).iloc[:, 1:2]

other_diagnoses_cols = ['SUBJECT_ID']+[col for col in cad if 'other_diagnoses' in col]
cad['other_diagnoses_all'] = cad[other_diagnoses_cols].bfill(axis=1).iloc[:, 1:2]

prim = primary_pie = px.pie(session1, names='primary_diagnoses_all', title='Subject Primary Diagnoses')
oth = other_pie = px.pie(session1, names='other_diagnoses_all', title='Subject Other Diagnoses')

prim.show()

In [10]:
cad = surveys['clinical_administered_data'][2:]
cad['duration_mins'] = pd.to_numeric(cad['Duration (in seconds)'])/60
display(cad)

/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_23656/1096552471.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Unnamed: 0,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,ymrs_10,panss_p_total,panss_n_total,panss_g_total,madrs_total,Create New Field or Choose From Dropdown...,ymrs_total,primary_diagnoses_all,other_diagnoses_all,duration_mins
2,2,2025-06-06 15:10:05,2025-06-06 15:46:08,IP Address,130.219.10.91,100,2162,True,2025-06-06 15:46:09,R_71ueZczuWAgImQQ,...,0 = Appropriate dress and grooming,1++1+1+33+33,33+1+33++33+33+33,33+33+33+33+33++33+33+33,1+1+1+1+1+1+1+1+11,NaN,+0+0+0+0+0+0+0+0+0+0,Healthy control,NaN,36.033333
3,3,2025-06-06 18:28:04,2025-06-06 18:39:10,IP Address,130.219.10.91,100,666,True,2025-06-06 18:39:11,R_5dELLaScSz3Pvfn,...,0 = Appropriate dress and grooming,1++1+1+33+33,33+1+33+33+33+33+33,33+2+33+33+33+1+33+33+33,0+0+1+1+1+1+1+1+10,NaN,1+0+0+0+0+0+0+0+0+0+0,Healthy control,NaN,11.100000
4,4,2025-06-10 19:17:09,2025-06-10 20:12:09,IP Address,130.219.10.91,100,3299,True,2025-06-10 20:12:09,R_7QYoz5e9Fcg32we,...,0 = Appropriate dress and grooming,2++2+1+33+33,33+3+33+2+2+33+33,2+2+2+33+33+2+33+33+33,1+1+0+2+1+1+1+1+10,NaN,1+2+0+1+0+0+0+0+0+0+0,Bipolar w/ psychotic features,Substance use disorder: Alcohol,54.983333
5,5,2025-06-13 20:47:43,2025-06-13 21:20:12,IP Address,130.219.10.91,100,1948,True,2025-06-13 21:20:12,R_1CkgUUiDZ5ytQ4V,...,0 = Appropriate dress and grooming,1++1+1+33+33,33+1+33+33+33+33+33,33+2+33+33+33+33+33+33+33,0+0+2+0+0+0+0+0+00,NaN,1+0+0+0+0+0+0+0+0+0+0,Healthy control,NaN,32.466667
6,6,2025-06-13 21:36:47,2025-06-13 22:42:28,IP Address,67.81.84.35,100,3941,True,2025-06-13 22:42:28,R_5PaQpCvbznucVQK,...,0 = Appropriate dress and grooming,1++1+1+33+33,33+1+33+33+2+33+33,2+3+33+33+33+33+33+33+33,0+0+0+0+0+0+0+0+00,NaN,0+0+0+0+0+0+0+0+0+0+0,Bipolar disorder,PTSD/Adjustment Disorder/Other specified traum...,65.683333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,58,2025-09-12 19:40:48,2025-09-12 21:25:56,IP Address,128.6.147.90,100,6307,True,2025-09-12 21:25:57,R_6d3UBAbu2ky6bqG,...,0 = Appropriate dress and grooming,1++1+1+4+1,3+1+1+3+1+1+1,1+5+1+3+1+5+1+1+1,3+4+4+4+2+3+2+2+30,NaN,0+0+2+0+2+0+1+0+2+0+2,Bipolar disorder,NaN,105.116667
59,59,2025-09-17 14:29:13,2025-09-17 15:08:18,IP Address,130.219.10.91,100,2344,True,2025-09-17 15:08:19,R_3D1R9RbsrNiuIGw,...,0 = Appropriate dress and grooming,1++1+1+1+1,1+4+1+1+2+1+1,1+3+3+1+1+3+1+1+1,0+2+3+0+0+3+2+0+30,NaN,0+0+0+0+0+0+0+0+0+0+0,Unipolar depression,NaN,39.066667
60,60,2025-09-17 19:48:41,2025-09-17 20:08:33,IP Address,132.183.13.119,100,1191,True,2025-09-17 20:08:34,R_6IZiOXPGe3xQTOH,...,0 = Appropriate dress and grooming,1++1+1+1+1,1+1+1+1+1+1+1,1+4+1+2+1+5+1+1+1,4+4+3+5+3+4+3+4+35,NaN,0+0+0+1+0+0+0+0+0+0+0,Unipolar depression,Generalized anxiety disorder (GAD),19.850000
61,61,2025-09-18 21:09:22,2025-09-18 22:39:43,IP Address,132.183.13.119,100,5420,True,2025-09-18 22:39:43,R_1lcCoxqFnK1oqyt,...,0 = Appropriate dress and grooming,2++2+2+2+2,2+1+1+1+1+1+1,3+5+4+3+2+4+1+1+2,2+4+4+6+3+3+2+3+32,NaN,1+0+0+0+0+0+0+0+0+0+1,Unipolar depression,PTSD/Adjustment Disorder/Other specified traum...,90.333333


In [ ]:

# Diagnoses
cad = surveys['clinical_administered_data'][2:]
prim_diagnoses_cols = ['SUBJECT_ID']+[col for col in cad if 'primary_diagnoses' in col]


/var/folders/fl/b24z_8kn4490x_bl0njv6fg00000gn/T/ipykernel_23656/4288274636.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Unnamed: 0,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,ymrs_09,ymrs_10,panss_p_total,panss_n_total,panss_g_total,madrs_total,Create New Field or Choose From Dropdown...,ymrs_total,primary_diagnoses_all,other_diagnoses_all
2,2,2025-06-06 15:10:05,2025-06-06 15:46:08,IP Address,130.219.10.91,100,2162,True,2025-06-06 15:46:09,R_71ueZczuWAgImQQ,...,"0 = Absent, cooperative",0 = Appropriate dress and grooming,1++1+1+33+33,33+1+33++33+33+33,33+33+33+33+33++33+33+33,1+1+1+1+1+1+1+1+11,NaN,+0+0+0+0+0+0+0+0+0+0,Healthy control,NaN
3,3,2025-06-06 18:28:04,2025-06-06 18:39:10,IP Address,130.219.10.91,100,666,True,2025-06-06 18:39:11,R_5dELLaScSz3Pvfn,...,"0 = Absent, cooperative",0 = Appropriate dress and grooming,1++1+1+33+33,33+1+33+33+33+33+33,33+2+33+33+33+1+33+33+33,0+0+1+1+1+1+1+1+10,NaN,1+0+0+0+0+0+0+0+0+0+0,Healthy control,NaN
4,4,2025-06-10 19:17:09,2025-06-10 20:12:09,IP Address,130.219.10.91,100,3299,True,2025-06-10 20:12:09,R_7QYoz5e9Fcg32we,...,"0 = Absent, cooperative",0 = Appropriate dress and grooming,2++2+1+33+33,33+3+33+2+2+33+33,2+2+2+33+33+2+33+33+33,1+1+0+2+1+1+1+1+10,NaN,1+2+0+1+0+0+0+0+0+0+0,Bipolar w/ psychotic features,Substance use disorder: Alcohol
5,5,2025-06-13 20:47:43,2025-06-13 21:20:12,IP Address,130.219.10.91,100,1948,True,2025-06-13 21:20:12,R_1CkgUUiDZ5ytQ4V,...,"0 = Absent, cooperative",0 = Appropriate dress and grooming,1++1+1+33+33,33+1+33+33+33+33+33,33+2+33+33+33+33+33+33+33,0+0+2+0+0+0+0+0+00,NaN,1+0+0+0+0+0+0+0+0+0+0,Healthy control,NaN
6,6,2025-06-13 21:36:47,2025-06-13 22:42:28,IP Address,67.81.84.35,100,3941,True,2025-06-13 22:42:28,R_5PaQpCvbznucVQK,...,"0 = Absent, cooperative",0 = Appropriate dress and grooming,1++1+1+33+33,33+1+33+33+2+33+33,2+3+33+33+33+33+33+33+33,0+0+0+0+0+0+0+0+00,NaN,0+0+0+0+0+0+0+0+0+0+0,Bipolar disorder,PTSD/Adjustment Disorder/Other specified traum...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,58,2025-09-12 19:40:48,2025-09-12 21:25:56,IP Address,128.6.147.90,100,6307,True,2025-09-12 21:25:57,R_6d3UBAbu2ky6bqG,...,"2 = Sarcastic; loud at times, guarded",0 = Appropriate dress and grooming,1++1+1+4+1,3+1+1+3+1+1+1,1+5+1+3+1+5+1+1+1,3+4+4+4+2+3+2+2+30,NaN,0+0+2+0+2+0+1+0+2+0+2,Bipolar disorder,NaN
59,59,2025-09-17 14:29:13,2025-09-17 15:08:18,IP Address,130.219.10.91,100,2344,True,2025-09-17 15:08:19,R_3D1R9RbsrNiuIGw,...,"0 = Absent, cooperative",0 = Appropriate dress and grooming,1++1+1+1+1,1+4+1+1+2+1+1,1+3+3+1+1+3+1+1+1,0+2+3+0+0+3+2+0+30,NaN,0+0+0+0+0+0+0+0+0+0+0,Unipolar depression,NaN
60,60,2025-09-17 19:48:41,2025-09-17 20:08:33,IP Address,132.183.13.119,100,1191,True,2025-09-17 20:08:34,R_6IZiOXPGe3xQTOH,...,"0 = Absent, cooperative",0 = Appropriate dress and grooming,1++1+1+1+1,1+1+1+1+1+1+1,1+4+1+2+1+5+1+1+1,4+4+3+5+3+4+3+4+35,NaN,0+0+0+1+0+0+0+0+0+0+0,Unipolar depression,Generalized anxiety disorder (GAD)
61,61,2025-09-18 21:09:22,2025-09-18 22:39:43,IP Address,132.183.13.119,100,5420,True,2025-09-18 22:39:43,R_1lcCoxqFnK1oqyt,...,"0 = Absent, cooperative",0 = Appropriate dress and grooming,2++2+2+2+2,2+1+1+1+1+1+1,3+5+4+3+2+4+1+1+2,2+4+4+6+3+3+2+3+32,NaN,1+0+0+0+0+0+0+0+0+0+1,Unipolar depression,PTSD/Adjustment Disorder/Other specified traum...


In [ ]:
sub = 'PCT929'
df = accel_df.query("subject_id == @sub")
xr_accel = xr.DataArray(df[activityScores].values, 
					   dims=["rows", "columns"],
					   coords={"rows": df['day'], "columns": activityScores})
fig = px.imshow(xr_accel, origin='lower', title='Activity Scores for PCT927',
				zmin=0,zmax=5,height=400, width=600)
fig.show()

In [ ]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output

app = Dash()
df = accel_df

# App layout
app.layout = [
	html.Div(children='Accelerometer Data'),
	dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
		options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
	dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
	Output(component_id='controls-and-graph', component_property='figure'),
	Input(component_id='subject_id', component_property='value')
)


def cb(subject_id):
	"""Callback to update the figure based on the selected id"""
	sub = subject_id
	df = accel_df.query("subject_id == @sub")
	xr_accel = xr.DataArray(df[activityScores].values, 
						dims=["rows", "columns"],
						coords={"rows": df['day'], "columns": activityScores})
	fig = px.imshow(xr_accel, origin='lower', title=f'Activity Scores for sub-{sub}',
					zmin=0,zmax=5,height=400, width=600)
	return fig

if __name__ == '__main__':
	app.run(mode='inline',port=8052)  # or 'external' if you want it in a new tab

In [ ]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output

app = Dash()

mindlamp_data = os.path.join(project_dir, 'mindlamp_data','processed_data')

power = {}
for sub in os.listdir(mindlamp_data):
	if os.path.exists(os.path.join(mindlamp_data,sub,'processed','power')):
		for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','power')):
			if 'phone_power_activityScores_hourly' in file:
				power[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','power',file))
				power[sub]['subject_id'] = sub

activityScores = [col for col in df.columns if 'activityScore' in col]
power_df = pd.concat([sub for sub in power.values()], axis=0, ignore_index=True)

df = power_df

# App layout
app.layout = [
	html.Div(children='Phone Use Data'),
	dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
		options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
	dcc.RadioItems(id='days', value='percentHome', 
		options=['All days','Weekdays','Weekends']),

	dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
	Output(component_id='controls-and-graph', component_property='figure'),
	Input(component_id='subject_id', component_property='value'),
	Input(component_id='days', component_property='value')
)


def cb(subject_id, days):
	"""Callback to update the figure based on the selected id"""
	sub = subject_id
	df = power_df.query("subject_id == @sub")
	if days == 'Weekdays':
		df = df[df['weekday'].astype(str).str.contains('1|2|3|4|5')]
	elif days == 'Weekends':
		df = df[df['weekday'].astype(str).str.contains('6|7')]
	xr_power = xr.DataArray(df[activityScores].values, 
						dims=["rows", "columns"],
						coords={"rows": df['day'], "columns": activityScores})
	fig = px.imshow(xr_power, origin='lower', title=f'Phone Activity (minutes each hour) for sub-{sub}',
					zmin=0,zmax=60,height=400, width=600)
	return fig

if __name__ == '__main__':
	app.run(mode='inline',port=8053)  # or 'external' if you want it in a new tab

In [ ]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output

app = Dash()

mindlamp_data = os.path.join(project_dir, 'mindlamp_data','processed_data')

gps = {}
for sub in os.listdir(mindlamp_data):
	if os.path.exists(os.path.join(mindlamp_data,sub,'processed','gps')):
		for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','gps')):
			if 'phone_gps_freq_hourly' in file:
				gps[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','gps',file))
				gps[sub]['subject_id'] = sub

activityScores = [col for col in df.columns if 'activityScore' in col]
gps_df = pd.concat([sub for sub in gps.values()], axis=0, ignore_index=True)

df = gps_df

# App layout
app.layout = [
	html.Div(children='GPS Data'),
	dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
		options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
	dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
	Output(component_id='controls-and-graph', component_property='figure'),
	Input(component_id='subject_id', component_property='value')
)


def cb(subject_id):
	"""Callback to update the figure based on the selected id"""
	sub = subject_id
	df = gps_df.query("subject_id == @sub")
	xr_gps = xr.DataArray(df[activityScores].values, 
						dims=["rows", "columns"],
						coords={"rows": df['day'], "columns": activityScores})
	fig = px.imshow(xr_gps, origin='lower', title=f'GPS Activity per Hour for sub-{sub}',
					zmin=0,zmax=5,height=400, width=600)
	return fig

if __name__ == '__main__':
	app.run(mode='inline',port=8053)  # or 'external' if you want it in a new tab

In [ ]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output

app = Dash()

mindlamp_data = os.path.join(project_dir, 'mindlamp_data','processed_data')

gps = {}
for sub in os.listdir(mindlamp_data):
	if os.path.exists(os.path.join(mindlamp_data,sub,'processed','gps')):
		for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','gps')):
			if 'phone_gps_homeStay_daily' in file:
				gps[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','gps',file))
				gps[sub]['subject_id'] = sub

gps_df = pd.concat([sub for sub in gps.values()], axis=0, ignore_index=True)

# App layout
app.layout = [
	html.Div(children='Mobility Information for Participants'),
	dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
		options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
	dcc.RadioItems(id='gps_item', value='percentHome', 
		options=['radiusMobility','percentHome','numPlaces']),
	dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
	Output(component_id='controls-and-graph', component_property='figure'),
	Input(component_id='subject_id', component_property='value'),
	Input(component_id='gps_item', component_property='value')
)


def cb(subject_id, gps_item):
	"""Callback to update the figure based on the selected id"""
	sub = subject_id
	sub_df = gps_df.query("subject_id == @sub")
	fig = px.line(sub_df, x="day", y=gps_item, title=f'{gps_item} for sub-{sub}')
	return fig

if __name__ == '__main__':
	app.run(mode='inline',port=8055)  # or 'external' if you want it in a new tab

In [ ]:
import dash
from dash import Dash, html, dash_table, dcc, callback, Input, Output
import dash_bootstrap_components as dbc
activity_app = Dash()
mindlamp_data = os.path.join(project_dir, 'mindlamp_data','processed_data')

accel = {}
for sub in os.listdir(mindlamp_data):
	if os.path.exists(os.path.join(mindlamp_data,sub,'processed','accel')):
		for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','accel')):
			if 'phone_accel_activity' in file:
				accel[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','accel',file))
				accel[sub]['subject_id'] = sub
accel_df = pd.concat([sub for sub in accel.values()], axis=0, ignore_index=True)

activityScores = [col for col in accel_df.columns if 'activityScore' in col]

gps = {}
for sub in os.listdir(mindlamp_data):
	if os.path.exists(os.path.join(mindlamp_data,sub,'processed','gps')):
		for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','gps')):
			if 'phone_gps_homeStay_daily' in file:
				gps[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','gps',file))
				gps[sub]['subject_id'] = sub

gps_df = pd.concat([sub for sub in gps.values()], axis=0, ignore_index=True)

# App layout
activity_app.layout = dbc.Container[
	html.H1(children='Accelerometer Data'),
	dcc.Dropdown(id='subject_id_input', value='PCT927', clearable=False,
		options=[{"label": f'sub-{y}', "value": y} for y in accel_df['subject_id'].unique()]),
	dcc.Graph(figure={}, id='accel_graph')
	
	# html.Div(children='Mobility Information for Participants'),
	# dcc.Dropdown(id="subject_id_input", value='PCT927', clearable=False,
	#     options=[{"label": f'sub-{y}', "value": y} for y in gps_df['subject_id'].unique()]),
	# dcc.RadioItems(id='gps_item', value='percentHome', 
	#     options=['radiusMobility','percentHome','numPlaces']),
	# dcc.Graph(figure={}, id='gps-graph')
]

# Add controls to build the interaction
@callback(
	Output(component_id='accel_graph', component_property='figure'),
	#Output(component_id='gps-graph', component_property='figure'),
	Input(component_id='subject_id_input', component_property='value'),
	Input(component_id='gps_item', component_property='value')
)


def cb(subject_id_input, gps_item):
	"""Callback to update the figure based on the selected id"""
	sub = subject_id_input
	accel_sub = accel_df.query("subject_id == @sub")
	accel_xr = xr.DataArray(accel_sub[activityScores].values, 
						dims=["rows", "columns"],
						coords={"rows": accel_sub['day'], "columns": activityScores})
	accel_fig = px.imshow(accel_xr, origin='lower', title=f'Activity Scores for sub-{sub}',
					zmin=0,zmax=5,height=400, width=600)
	
	# gps_sub = gps_df.query("subject_id == @sub")
	# gps_fig = px.line(gps_sub, x="day", y=gps_item, title=f'{gps_item} for sub-{sub}')

	return accel_fig


if __name__ == '__main__':
	activity_app.run(mode='inline',port=8052)  # or 'external' if you want it in a new tab

TypeError: type 'Container' is not subscriptable

In [ ]:
# Initialize the app - incorporate css
import dash_bootstrap_components as dbc

external_stylesheets = [dbc.themes.CERULEAN]
gps_app = Dash(__name__, external_stylesheets=external_stylesheets)

gps = {}
for sub in os.listdir(mindlamp_data):
	if os.path.exists(os.path.join(mindlamp_data,sub,'processed','gps')):
		for file in os.listdir(os.path.join(mindlamp_data,sub,'processed','gps')):
			if 'phone_gps_homeStay_daily' in file:
				gps[sub] = pd.read_csv(os.path.join(mindlamp_data,sub,'processed','gps',file))
				gps[sub]['subject_id'] = sub

gps_df = pd.concat([sub for sub in gps.values()], axis=0, ignore_index=True)

# App layout
gps_app.layout = [
	html.Div(children='Mobility Information for Participants'),
	dcc.Dropdown(id="subject_id", value='PCT927', clearable=False,
		options=[{"label": f'sub-{y}', "value": y} for y in df['subject_id'].unique()]),
	dcc.RadioItems(id='gps_item', value='radiusMobility', 
		options=['radiusMobility','percentHome','numPlaces']),
	dcc.Graph(figure={}, id='controls-and-graph')
]

# Add controls to build the interaction
@callback(
	Output(component_id='controls-and-graph', component_property='figure'),
	Input(component_id='subject_id', component_property='value'),
	Input(component_id='gps_item', component_property='value')
)


def cb(subject_id, gps_item):
	"""Callback to update the figure based on the selected id"""
	sub = subject_id
	gps_sub = gps_df.query("subject_id == @sub")
	gps_fig = px.line(gps_sub, x="day", y=gps_item, title=f'{gps_item} for sub-{sub}')
	return gps_fig

if __name__ == '__main__':
	gps_app.run(mode='inline',port=8054)  # or 'external' if you want it in a new tab